In [1]:
import datacube
from odc.ui import with_ui_cbk

In [2]:
dc = datacube.Datacube(app='Sentinel_2')

In [3]:
ds = dc.load(product="s2_l2a",
             measurements=['blue','green','red','nir'],
             y=(-9, -5),
             x=(43.14, 42.56),
             time=("2022-01-01", "2022-01-31"),
             resolution=(-60, 60),
             output_crs='EPSG:4258',
             group_by="solar_day")

print(ds)

<xarray.Dataset>
Dimensions:      (latitude: 1, longitude: 1, time: 6)
Coordinates:
  * time         (time) datetime64[ns] 2022-01-02T07:41:07 ... 2022-01-27T07:...
  * latitude     (latitude) float64 -30.0
  * longitude    (longitude) float64 30.0
    spatial_ref  int32 4258
Data variables:
    blue         (time, latitude, longitude) uint16 0 0 0 0 0 0
    green        (time, latitude, longitude) uint16 0 0 0 0 0 0
    red          (time, latitude, longitude) uint16 0 0 0 0 0 0
    nir          (time, latitude, longitude) uint16 0 0 0 0 0 0
Attributes:
    crs:           EPSG:4258
    grid_mapping:  spatial_ref
